In [1]:
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
import numpy as np
import os

In [3]:
def create_model(opt):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),                         
        tf.keras.layers.Dense(64, activation='relu', name='dense_1'),
        tf.keras.layers.Dense(64, activation='relu', name='dense_2'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

    return model

In [4]:

#Load Fashion MNIST dataset
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

test_images, test_labels = test

In [5]:
#Optimizers 
sgd = tf.keras.optimizers.SGD(0.01)
stocastic_avg_sgd = tfa.optimizers.SWA(sgd)

In [6]:
checkpoint_path = "./training/model.ckpt"

avg_callback = tfa.callbacks.AverageModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,
                                                    update_weights=True)

In [7]:
#Build Model
model = create_model(stocastic_avg_sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[avg_callback])

Train for 1875 steps
Epoch 1/5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7908 - accuracy: 0.7343
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5668 - accuracy: 0.8081
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5360 - accuracy: 0.8167
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5208 - accuracy: 0.8217
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5105 - accuracy: 0.8253


In [8]:
#Evalute results
model.load_weights(checkpoint_path)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

10000/10000 - 0s - loss: 72.3007 - accuracy: 0.7759
Loss : 72.30067407073975
Accuracy : 0.7759
